<a href="https://colab.research.google.com/github/wockeshuh/CSCI166/blob/main/Q_learning_FrozenLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Frozen Lake Domain Description

Frozen Lake is a simple grid-world environment where an agent navigates a frozen lake to reach a goal while avoiding falling into holes. The environment is represented as a grid, with each cell being one of the following:

* **S**: Starting position of the agent
* **F**: Frozen surface, safe to walk on
* **H**: Hole, falling into one ends the episode with a reward of 0
* **G**: Goal, reaching it ends the episode with a reward of 1

The agent can take four actions:

* **0: Left**
* **1: Down**
* **2: Right**
* **3: Up**

However, due to the slippery nature of the ice, the agent might not always move in the intended direction. There's a chance it moves perpendicular to the intended direction.




In [16]:
import gym

# Create the environment
env = gym.make('FrozenLake-v1', render_mode='ansi')  # 'ansi' mode for text-based rendering

# Reset the environment to the initial state
observation = env.reset()

# Take a few actions and observe the results
for _ in range(5):
    action = env.action_space.sample()  # Choose a random action
    observation, reward, done, info = env.step(action)
    # Render the environment to see the agent's movement (text-based)
    if done:
        observation= env.reset()
    else:
      rendered = env.render()
      if len(rendered) > 1:  # Check if there's a second element
         print(rendered[1])  # Print the second element
# Close the environment
env.close()

  (Up)
SFFF
FHFH
FFFH
HFFG

  (Right)
SFFF
FHFH
FFFH
HFFG



/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


The transition model for the Frozen Lake world describes how the agent's actions affect its movement and the resulting state transitions. Here's a breakdown of the key components:

**Actions:**

* The agent can choose from four actions:
    * 0: Left
    * 1: Down
    * 2: Right
    * 3: Up

**State Transitions:**

* **Intended Movement:** Ideally, the agent moves one cell in the chosen direction.
* **Slippery Ice:** Due to the slippery nature of the ice, there's a probability that the agent will move in a perpendicular direction instead of the intended one. The exact probabilities depend on the specific Frozen Lake environment configuration, but typically:
    * **Successful Move:** The agent moves in the intended direction with a high probability.
    * **Perpendicular Move:** The agent moves 90 degrees to the left or right of the intended direction with a lower probability.
* **Boundaries:** If the intended movement would take the agent outside the grid boundaries, it remains in its current position.
* **Holes:** If the agent lands on a hole ("H"), the episode ends, and it receives a reward of 0.
* **Goal:** If the agent reaches the goal ("G"), the episode ends, and it receives a reward of 1.




In [17]:
import gym

# Create the environment
env = gym.make('FrozenLake-v1', render_mode='ansi')  # 'ansi' mode for text-based rendering

# Reset the environment to the initial state
observation = env.reset()

# Take a few actions and observe the results
for _ in range(5):
    action = env.action_space.sample()  # Choose a random action
    observation, reward, done, info = env.step(action)
    # Render the environment to see the agent's movement (text-based)
    if done:
        observation= env.reset()
    else:
      rendered = env.render()
      if len(rendered) > 1:  # Check if there's a second element
         print(rendered[1])  # Print the second element
# Close the environment
env.close()
print ("State 14 Going Right: (s, a, r, Done)", env.P[14][2])

  (Down)
SFFF
FHFH
FFFH
HFFG

  (Up)
SFFF
FHFH
FFFH
HFFG

State 14 Going Right: (s, a, r, Done) [(0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False)]


In [18]:
import gym
import numpy as np

# Create FrozenLake environment
env = gym.make("FrozenLake-v1")

# Starter code for students (modified for number of iterations)
def value_iteration(env, gamma=0.9, num_iterations=1000):
    """
    Implements the Value Iteration algorithm.

    Args:
        env: The OpenAI Gym environment.
        gamma: Discount factor.
        num_iterations: Number of iterations to run.

    Returns:
        The optimal value function and policy.
    """

    # Initialize value function and policy
    V = np.zeros(env.observation_space.n)
    policy = np.zeros(env.observation_space.n)

    # TODO: Implement the core Value Iteration logic here
    # - Iterate for 'num_iterations'
    # - For each state:
    #   - Calculate Q values for all actions
    #   - Update V[s] with the max Q value
    #   - Update policy[s] with the action that maximizes Q value

    for _ in range(num_iterations):
        for s in range(env.observation_space.n):
            q_values = np.zeros(env.action_space.n)
            for a in range(env.action_space.n):
                for prob, next_state, reward, done in env.P[s][a]:
                    q_values[a] += prob * (reward + gamma * V[next_state])
            V[s] = np.max(q_values)
            policy[s] = np.argmax(q_values)

    return V, policy


# Apply student's Value Iteration
optimal_V, optimal_policy = value_iteration(env)

# Evaluate student's solution (Optional)
def evaluate_policy(env, policy, num_episodes=100):
    total_reward = 0
    for _ in range(num_episodes):
        state = env.reset()
        done = False
        while not done:
            action = policy[state]
            state, reward, done, _ = env.step(action)
            total_reward += reward
    return total_reward / num_episodes

average_reward = evaluate_policy(env, optimal_policy)
print("Optimal Value:")
print(optimal_V.reshape(4,4))
print("Optimal Policy:")
print(optimal_policy.reshape(4,4))
print("Average Reward:", average_reward)

Optimal Value:
[[0.0688909  0.06141457 0.07440976 0.05580732]
 [0.09185454 0.         0.11220821 0.        ]
 [0.14543635 0.24749695 0.29961759 0.        ]
 [0.         0.3799359  0.63902015 0.        ]]
Optimal Policy:
[[0. 3. 0. 3.]
 [0. 0. 0. 0.]
 [3. 1. 0. 0.]
 [0. 2. 1. 0.]]
Average Reward: 0.74


# Q-Learning Algorithm

In [19]:
import gym
import numpy as np

env = gym.make("FrozenLake-v1")

def q_learning(env, gamma=0.9, alpha=0.8, epsilon=0.1, num_episodes=10000):
    Q = np.zeros([env.observation_space.n, env.action_space.n])

    for _ in range(num_episodes):
        state = env.reset()
        done = False
        while not done:
            # Epsilon-Greedy action selection
            if np.random.rand() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(Q[state])
            next_state, reward, done, _ = env.step(action)

            # Q-Learning update
            Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[next_state]) - Q[state, action])
            state = next_state

    # Extract optimal policy from Q-value function
    policy = np.argmax(Q, axis=1)

     return Q, policy

   Q, q_policy = q_learning(env)

   print("Optimal Q-value:")
   print(Q)
   print("Optimal policy:")
   print(q_policy)

def evaluate_policy(env, policy, num_episodes=100):
    total_reward = 0
    for _ in range(num_episodes):
        state = env.reset()
        done = False
        while not done:
            action = policy[state]
            state, reward, done, _ = env.step(action)
            total_reward += reward
    return total_reward / num_episodes

  q_average_reward = evaluate_policy(env, q_policy)
  print("Average reward:", q_average_reward)

Optimal Q-value:
[[1.98254195e-01 4.71453074e-02 3.90545823e-02 6.41041617e-02]
 [7.26987785e-06 1.50121367e-02 4.34277177e-04 7.24105753e-02]
 [6.18277696e-02 1.01308981e-02 9.59322092e-03 9.08654910e-03]
 [2.33829200e-03 2.18387962e-03 7.31026210e-03 9.24939417e-03]
 [2.79336169e-01 3.85900444e-03 1.74680319e-02 1.78611829e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.75024213e-02 1.31538791e-07 4.12187912e-03 4.39108588e-07]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.39744736e-02 2.90466245e-02 3.84246691e-02 3.84886345e-01]
 [3.56508735e-02 2.50165649e-01 1.03099289e-01 2.25410923e-02]
 [7.64802333e-01 2.34471063e-04 3.33046453e-03 8.39743655e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.61882553e-02 1.50945349e-01 8.13639985e-01 5.06277433e-02]
 [2.40836731e-01 7.81935085e-01 9.83952618e-01 7.10466656e-01]
 [0.00000000e+00 0.00000000e+00 0.0000

In [23]:
print("Q-Learning vs Value Iteration policy difference:")
print((q_policy != optimal_policy).reshape(4, 4))
print("Q-Learning vs Value Iteration value difference:")
print((np.max(Q, axis=1) - optimal_V).reshape(4, 4))
print("Q-Learning Average reward:", q_average_reward)
print("Value Iteration Average reward:", average_reward)


Comparison:
Q-Learning vs Value Iteration policy difference:
[[False False False False]
 [False False False False]
 [False False False False]
 [False False  True False]]
Q-Learning vs Value Iteration value difference:
[[ 0.12936329  0.010996   -0.01258199 -0.04655793]
 [ 0.18748163  0.         -0.07470579  0.        ]
 [ 0.23944999  0.00266869  0.46518474  0.        ]
 [ 0.          0.43370408  0.34493247  0.        ]]
Q-Learning Average reward: 0.62
Value Iteration Average reward: 0.74


The policies for Q-learning algorithm and Value Iteration were the same for every state that has a False. There is only one state that is True and this represents where the two algorithm's policies diverge. From this, I think we can assume that the algorithms have similar approaches to solving a problem.

The difference in values show which states the algorithms favored. The positive values represent a state that Q-learning favored and negative values represent the states that Value Iteration favored. The Q-learning algorithm valued states that were more towards the left. This can be because it was trying to learn new actions. The Value Iteration algorithm favored the right side more. This can be because the goal is at the bottom right so it's trying to find the most optimal way there.

The average reward shows that Value Iteration is more efficient. I think this is because the Q-learning algorithm tries to solve a problem through trial and error.
